# Import librairies

In [ ]:
!pip3 install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu110/torch_nightly.html

In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
import string
import torch
import torch.nn as nn
import torch.nn.functional as F

import random
import numpy as np

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3060 Ti


# Import data

In [3]:
df = pd.read_csv("../data/training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None)

In [4]:
df = df.set_axis(["target","id","date","flag","user","text"], axis=1)


In [5]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
df = df.sample(5000)

# Pre-processing text

## Lowercase

In [7]:
df["text"] = df["text"].str.lower()

## Replace emojis with text

https://www.oreilly.com/library/view/python-cookbook/0596001673/ch03s15.html

https://medium.com/geekculture/text-preprocessing-how-to-handle-emoji-emoticon-641bbfa6e9e7

In [8]:
# 'Emoji_Dict.p'- download link https://drive.google.com/open?id=1G1vIkkbqPBYPKHcQ8qy0G2zkoab2Qv4v


with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}



# def convert_emojis_to_word(text):
#     regex = re.compile("|".join(map(re.escape, Emoji_Dict.keys(  ))))
#     # For each match, look up the corresponding value in the dictionary
#     text = regex.sub(lambda match: " ".join(Emoji_Dict[match.group(0).replace("_"," ").replace(",","").replace(":","").split()]), text)
#     text = re.sub("\s\s+" , " ", text)
#     # for emot in Emoji_Dict:
#     #     text = re.sub(emot,r' '+emot, text)
#     #     text = re.sub("\s\s+" , " ", text)
#     #     text = re.sub(r'('+emot+')', " ".join(Emoji_Dict[emot].replace("_"," ").replace(",","").replace(":","").split()), text)
#     return text

def convert_emojis_to_word(text):
    # print(text)
    regex = re.compile("|".join(map(re.escape, Emoji_Dict.keys(  ))))
    # print(regex.sub(lambda match: Emoji_Dict[match.group(0)], text))
    # print(regex.sub(lambda match: Emoji_Dict[match.group(0)].replace("_"," ").replace(","," ").replace(":"," "), text))
    text = regex.sub(lambda match: Emoji_Dict[match.group(0)].replace("_"," ").replace(","," ").replace(":"," "), text)
    # text = regex.sub(lambda match: " ".join(Emoji_Dict[match.group(0)].replace("_"," ").replace(","," ").replace(":"," ").split()), text)
    text = re.sub("\s\s+" , " ", text)
    return text

df["text"] = df["text"].apply(convert_emojis_to_word)

## Replace abbreviations

In [9]:

abbr_dict={
    "what's":"what is",
    "what're":"what are",
    "who's":"who is",
    "who're":"who are",
    "where's":"where is",
    "where're":"where are",
    "when's":"when is",
    "when're":"when are",
    "how's":"how is",
    "how're":"how are",

    "i'm":"i am",
    "we're":"we are",
    "you're":"you are",
    "they're":"they are",
    "it's":"it is",
    "he's":"he is",
    "she's":"she is",
    "that's":"that is",
    "there's":"there is",
    "there're":"there are",

    "i've":"i have",
    "we've":"we have",
    "you've":"you have",
    "they've":"they have",
    "who've":"who have",
    "would've":"would have",
    "not've":"not have",

    "i'll":"i will",
    "we'll":"we will",
    "you'll":"you will",
    "he'll":"he will",
    "she'll":"she will",
    "it'll":"it will",
    "they'll":"they will",

    "isn't":"is not",
    "wasn't":"was not",
    "aren't":"are not",
    "weren't":"were not",
    "can't":"can not",
    "couldn't":"could not",
    "don't":"do not",
    "didn't":"did not",
    "shouldn't":"should not",
    "wouldn't":"would not",
    "doesn't":"does not",
    "haven't":"have not",
    "hasn't":"has not",
    "hadn't":"had not",
    "won't":"will not",
    
    'shoulda': 'should have',
    'gonna': 'going to', 
    'wanna': 'wanting to',
    "ain't": "is not",
    "wana": 'wanting to',
   
    
    'ngl': 'not going to lie',
    'idk': 'i do not know',
    'fyi': 'for your information',
    'tbh': 'to be honest',
    'asap': 'as soon as possible',
    'bbiab': 'be back in a bit',
    'bbl': 'be back later',
    'bbs': 'be back soon',
    'bf': 'boyfriend',
    'bff': 'best friend forever',
    'brb': 'be right back',
    'cya': 'see you',
    'faq': 'frequently asked questions',
    'ftw': 'for the win',
    'g2g': 'got to go',
    'gf': 'girlfriend',
    'gr8': 'great',
    'hru': 'how are you',
    'ight': 'alright',
    'imo': 'in my opinion',
    'imy': 'i miss you',
    'irl': 'in real life',
    'istg': 'i swear',
    'lmao': 'laughing',
    'lmk': 'let me know',
    'lol': 'laughing',
    'nvd': 'nevermind',
    'noob': 'amateur',
    ' np ': ' no problem ',
    'ofc': 'of course',
    'omg': 'i can not believe it',
    'rn': 'right now',
    'ttyl': 'talk to you later',
    ' u ': ' you ',
    'wym': 'what do you mean ?',
    ' y ': ' why ',
    'yw': 'you are welcome'
    
}


def replace_abbreviations(text):
    regex = re.compile("|".join(map(re.escape, abbr_dict.keys(  ))))
    text = regex.sub(lambda match: abbr_dict[match.group(0)], text)
    return text
df["text"] = df["text"].apply(replace_abbreviations)

## Handle punctiation

In [10]:
punctiations = {}
for elem in string.punctuation:
    punctiations[elem] = f" {elem} "
punctiations["..."] = " ... "
def handle_punctiation(text):
    regex = re.compile("|".join(map(re.escape, punctiations.keys(  ))))
    text = regex.sub(lambda match: punctiations[match.group(0)], text)
    text = re.sub("\s\s+" , " ", text)
    return text
df["text"] = df["text"].apply(handle_punctiation)

## Remove stopwords

In [11]:
def filter_preprocess(x):
    # Delete links
    x = " ".join(filter(lambda y: not(y.startswith("http")), x.split()))
    x = " ".join(filter(lambda y: not(y.endswith(".com")), x.split()))
    
    # Remove stop words
    x = " ".join(filter(lambda y: not(y in stopwords.words('english')), x.split()))
    
    x = " ".join(filter(lambda y: y[0]!="@", x.split()))
    
    return x

df["text"] = df["text"].apply(filter_preprocess)

## Handle label

In [12]:
df["target"] = df["target"].apply(lambda x: int(x/4))

# Tokenize and encode words

In [13]:
from tqdm import tqdm
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [14]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# Create empty lists to store outputs
global input_ids
input_ids = []
global attention_masks
attention_masks = []
MAX_LEN = 100
# Create a function to tokenize a set of texts
def preprocessing_for_bert(out,x):
    """Perform required preprocessing steps for pretrained BERT.
    @param    df (dataframe): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    
    # `encode_plus` will:
    #    (1) Tokenize the sentence
    #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
    #    (3) Truncate/Pad sentence to max length
    #    (4) Map tokens to their IDs
    #    (5) Create attention mask
    #    (6) Return a dictionary of outputs
    x = tokenizer.encode_plus(
        x,
        max_length=MAX_LEN,                  # Max length to truncate/pad
        pad_to_max_length=True, 
        add_special_tokens=True,   
        # pad_to_max_length=True,         # Pad sentence to max length
        #return_tensors='pt',           # Return PyTorch tensor
        return_attention_mask=True      # Return attention mask
        )  
    return x.get(out)
 

df["ids"] = df["text"].apply(lambda x:  preprocessing_for_bert('input_ids',x))
df["masks"] = df["text"].apply(lambda x:  preprocessing_for_bert('attention_mask',x))
# Convert lists to tensors
# input_ids = torch.tensor(input_ids)
# attention_masks = torch.tensor(attention_masks)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [15]:
df.head()

,target,id,date,flag,user,text,ids,masks
1370221,1,2050976244,Fri Jun 05 19:38:09 PDT 2009,NO_QUERY,rgoodchild,adriennerewi agree - seeing awful lot fiscal r...,"[101, 4748, 23144, 3678, 7974, 2072, 5993, 101...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
128881,0,1834991215,Mon May 18 04:49:48 PDT 2009,NO_QUERY,OuGee,otw grannys love ( westlife ) playing car . wtf .,"[101, 27178, 2860, 19794, 2015, 2293, 1006, 22...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1144810,1,1977673971,Sat May 30 21:27:31 PDT 2009,NO_QUERY,lovesodangerous,fatgin real lifesguide ! rule .,"[101, 6638, 11528, 2613, 2166, 28745, 21272, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..."
1248573,1,1995853825,Mon Jun 01 14:09:00 PDT 2009,NO_QUERY,KluvsNKOTB,joeymcintyre wait see guys weeks,"[101, 9558, 12458, 18447, 16363, 3524, 2156, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
545549,0,2201470329,Tue Jun 16 20:19:12 PDT 2009,NO_QUERY,impossiblecat,bookiemonsteright nowz publishers chose even k...,"[101, 2338, 2666, 16563, 3334, 18743, 2085, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


# Train/test split

In [16]:
from sklearn.model_selection import train_test_split

# X = df["text"]
# y = df["target"]
train_inputs, val_inputs, train_masks, val_masks,  y_train, y_val = train_test_split(df["ids"],df["masks"], df["target"], test_size=0.2, random_state=202)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=202)

# Dataset to feed to NN

In [17]:
len(train_masks.to_numpy()[3])

100

In [18]:
np.array(train_masks.to_list()).shape

(4000, 100)

In [19]:
np.array(train_inputs.to_list()).shape

(4000, 100)

In [21]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train.to_numpy())
val_labels = torch.tensor(y_val.to_numpy())

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset( torch.tensor(np.array(train_inputs.to_list())), torch.tensor(np.array(train_masks.to_list())), train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(torch.tensor(np.array(val_inputs.to_list())), torch.tensor(np.array(val_masks.to_list())), val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

# Model

In [22]:
from transformers import BertModel
# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [23]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [24]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [25]:
# !pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [26]:
print(torch.max(torch.rand((30,30),device='cuda')))

tensor(0.9978, device='cuda:0')


In [27]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.681343   |     -      |     -     |   6.38   
   1    |   40    |   0.629627   |     -      |     -     |   5.27   
   1    |   60    |   0.530610   |     -      |     -     |   5.39   
   1    |   80    |   0.586305   |     -      |     -     |   5.35   
   1    |   100   |   0.554632   |     -      |     -     |   5.29   
   1    |   120   |   0.547318   |     -      |     -     |   5.36   
   1    |   124   |   0.479586   |     -      |     -     |   1.08   
----------------------------------------------------------------------
   1    |    -    |   0.585571   |  0.476179  |   77.64   |   36.06  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------------

# Experimental (stemming)

In [ ]:
from nltk.stem.snowball import SnowballStemmer

englishStemmer=SnowballStemmer("english")

In [ ]:
englishStemmer.keys()

In [ ]:
!pip3 install emot